In [1]:
import tensorflow as tf

```
本篇博客记录tensorflow中和数据处理相关的操作。
```

## 1. list of indices to one hot


比如原来的index列表是[0, 2, -1, 1]

In [4]:
one_hot = tf.one_hot(indices = [0, 2, -1, 1],
                     depth = 3,
                     on_value = 1.0,
                     off_value = 0.0,
                     axis = -1)

In [5]:
sess = tf.Session()
print sess.run(one_hot)

[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]
 [ 0.  1.  0.]]


这个操作很像embedding lookup，只是返回的数据是depth维的one hot向量。